# buy-open-sell-close

On the first trading day of each week, buy on the open, then sell on the close.

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# Fetch timeseries, select.
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# Add calendar columns.
ts = pf.calendar(ts)

# Finalize timeseries.
ts, start = pf.finalize_timeseries(ts, start)

# Create tradelog and daily balance objects.
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [4]:
ts

,open,high,low,close,adj_close,volume,dotw,dotm,doty,month,first_dotw,first_dotm,first_doty,last_dotw,last_dotm,last_doty
date,,,,,,,,,,,,,,,,
2015-01-02,206.38,206.88,204.18,205.43,172.59,121465900,4,2,2,1,False,True,True,True,False,False
2015-01-05,204.17,204.37,201.35,201.72,169.48,169632600,0,5,5,1,True,False,False,False,False,False
2015-01-06,202.09,202.72,198.86,199.82,167.88,209151400,1,6,6,1,False,False,False,False,False,False
2015-01-07,201.42,202.72,200.88,202.31,169.97,125346700,2,7,7,1,False,False,False,False,False,False
2015-01-08,204.01,206.16,203.99,205.90,172.99,147217800,3,8,8,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-20,581.77,595.75,580.91,591.15,591.15,125716700,4,20,355,12,False,False,False,True,False,False
2024-12-23,590.89,595.30,587.66,594.69,594.69,57635800,0,23,358,12,True,False,False,False,False,False
2024-12-24,596.06,601.34,595.47,601.30,601.30,33160100,1,24,359,12,False,False,False,False,False,False


Algorithm

In [5]:
pf.TradeLog.cash = capital

# Loop through timeseries.
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # On first trading day of the week, buy open, then sell close.
    if row.first_dotw:
        tlog.buy(date, row.open)
        tlog.sell(date, row.close)

    # Record daily balance.
    dbal.append(date, row.close, row.high, row.low)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
516,2024-11-25,599.52,2024-11-25,597.53,-1.99,-37.81,19,1715.36,LONG,SPY
517,2024-12-02,602.97,2024-12-02,603.63,0.66,12.54,19,1727.90,LONG,SPY
518,2024-12-09,607.69,2024-12-09,604.68,-3.01,-57.19,19,1670.71,LONG,SPY
519,2024-12-16,606.00,2024-12-16,606.79,0.79,15.01,19,1685.72,LONG,SPY
520,2024-12-23,590.89,2024-12-23,594.69,3.80,72.20,19,1757.92,LONG,SPY


Get stats

In [8]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [9]:
pf.summary(stats)

,strategy
annual_return_rate,1.64
max_closed_out_drawdown,-13.98
best_month,6.04
worst_month,-5.08
sharpe_ratio,0.34
sortino_ratio,0.21
monthly_std,1.37
annual_std,6.00
